# CS-5600/6600 Lecture 13 - Statistics

**Instructor: Dylan Zwick**

*Weber State University*

Reference: [Hands-On Machine Learning with Scikit-Learn, Keras & TensorFlow](https://www.oreilly.com/library/view/hands-on-machine-learning/9781098125967/) by Aurélien Géron - [Dimensionality Reduction](https://github.com/ageron/handson-ml3/blob/main/08_dimensionality_reduction.ipynb)

<center>
  <img src="https://imgs.xkcd.com/comics/correlation.png" alt="Correlation and Causation">
</center>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import csv

from scipy import stats

np.random.seed(6600)

Today, we're going to talk about statistics. My goal, of course, is not to provide anything close to all the statistics that you'll need for understanding machine learning. That would be an entire class on its own, and that still wouldn't be enough time! My goal instead is to introduce the foundational language that we'll all want to understand when we're talking about machine learning models, which are fundamentally based on concepts from statistics.

##The Dalton Heights Dataset

Around 1886 Francis Galton published a study in which he investigated the statistical relation between the height of a person and the height of that person's parents. This study is the basis for the term *regression* (which we'll discuss next time), and more specifically the term *regression towards the mean* (or "regression towards the mediocre" as Dalton put it.)

Let's check out the dataset he used:

In [ ]:
df = pd.read_csv('https://drive.google.com/uc?export=view&id=1fJoQp6ho1rkhuZUBNd39ThUMOCheDOD0')
df.head()

Here we see that for family, we have the height of the father, the height of the mother, the "midparent height", which is a predictive height Galton used defined by:

<center>
  $\displaystyle H = \frac{1}{2}(H_{father} + 1.08 H_{mother})$
</center>

We also have the number of children within the family, the order of the children, the gender of the children, and the heights of the children.

We'll revisit these data later, but for today let's just focus on the last column - the heights of the children. I'm going to take the values from this column and convert them into a numpy array so we can more effectively do math with them.

In [ ]:
heights = df['childHeight'].to_numpy()
heights

And, well, there they are. Those are the height values for all the children in our study. That's our data. If we wanted to visualize it a bit informatively than just printing out the array, we could do so with a histogram:

In [ ]:
plt.hist(df['childHeight'])
plt.xlabel('Height (cm)')
plt.ylabel('Frequency')
plt.title('Distribution of Child Heights from Galton Data')
plt.show()

## The Meaning of the Mean

But, suppose I wanted to communicate something about this data that summarized some key aspect of it without including everything. This is what we mean by a "[statistic](https://https://en.wikipedia.org/wiki/Statistic)". Probably the most popular statistic is the "mean" (or the average), which is what you get when you add up all the values, and divide by the number of values:

<center>
  $\displaystyle \frac{x_{1} + x_{2} + \cdots + x_{n}}{n}$
</center>

We can do this easily with numpy:

In [ ]:
mean_value = np.mean(heights)

print(f"The mean value of our data is: {mean_value:.2f}")

Now, the mean seems like a pretty obvious statistic, but why is it useful? What desirable properties does it have?

Well, we're focused on machine learning, which means we want to build predictive models. Suppose we wanted to build a predictive model for the heights, but we had to use the same prediction for everybody. So, it's just a constant model. What constant should we use?

Well, we'd want the one that minimized our error. But, how do we define our error? One way we could do it is just look at how far off we are - either over or under - and just add those up. So, for example, let's suppose we have five data points in our sample:

<center>
  $[5,6,7,8,9]$
</center>

Suppose our guess was $6$. In this case, our individual errors would be:

<center>
  $[-1, 0, 1, 2, 3]$
</center>

If we add these up, we get $5$. On the other hand, if our model's prediction were the mean - which in this case is $7$, then the individual errors would be:

<center>
  $[-2, -1, 0, 1, 2]$
</center>

We were to add these up - we'd get zero! Hooray! But this isn't really a good way to measure the success of our model. If we're looking at how wrong our predictions are, it's probably not good to say that two wrongs make a right. If you're off by 2 one way, and then off by 2 in the other way, that's not really as good as getting it exactly right twice. Is there a better way?

Sure. Instead of just looking at the difference between our prediction and the actual value, what we can do is look at the *square* of the difference. This number is minimized (equals $0$) if the prediction and the observation are exactly the same, and gets larger the farther off the predction is, regardless of direction (whether the prediction is too high or too low). We can write this error as an equation where $x_{1}, x_{2}, \ldots, x_{n}$ are our observations, and $\overline{x}$ is our prediction:

<center>
  Error = $\displaystyle (x_{1}-\overline{x})^{2} + (x_{2}-\overline{x})^{2} + \cdots + (x_{n}-\overline{x})^{2} = \sum_{i = 1}^{n} (x_{i}-\overline{x})^{2}$
</center>

What is the value of $\overline{x}$ that minimizes this error? Well, it's a straightforward calculus problem to derive that it's:

<center>
  $\displaystyle \overline{x} = \frac{\sum_{i = 1}^{n}x_{i}}{n}$
</center>

In other words - the mean! So, the mean is the constant value that gives us our best estimate, with the error measure above.

This error value will get larger with the number of observations. If we want to get some scaled measure of it, we can look at the average error per observation:

<center>
  $\displaystyle \frac{\sum_{i = 1}^{n}(x_{i}-\overline{x})^{2}}{n}$
</center>

This value is known as the *variance*, and it's another common statistic about our dataset. It can be viewed as a measure for how spread out our data is around its mean. It can also be easily calculated using methods from NumPy:

In [ ]:
var_value = np.var(heights)

print(f"Variance: {var_value:.2f}")

The variance has a different scale and, if the measurements are in units, different units than the observations and the mean. For example, if the observations are in inches, the variances in is square inches. This can be a bit tricky, and so instead of directly using the variance, we'll frequently take another statistic derived directly from the variance called the *standard deviation*, which is just the square root of the variance. It will have the same scale and units as the observations and the mean.

<center>
  Standard Deviation $\displaystyle = \sqrt{\frac{\sum_{i = 1}^{n}(x_{i}-\overline{x})^{2}}{n}}$
</center>

It can also be easily calculated with NumPy methods:

In [ ]:
std_dev_value = np.std(heights)

print(f"Standard Deviation: {std_dev_value:.2f}")

###The Median

Another popular statistic is the *median*, which like the mean is a statistic that attempts to measure the center of a distribution. The median is the observation in the center, such that half of the other observations are less, while half are more. If there are an even number of observations, then the median is the average of the two  observations closest to the center. It's less affected by outliers, and less straightforward as a mathematical equation, but can still be calculated with NumPy methods:

In [ ]:
median_value = np.median(heights)

print(f"Median: {median_value:.2f}")

Note that, generally speaking, the median is much harder to computer than the mean. This is because in order to calculate a mean, you just need to add up all the values in a list. In order to calculate a median, you need to *sort* all the values in a list, which is harder in the precise sense that it's in a different, more difficult complexity class. Adding up a list of numbers is $O(n)$, while sorting a list of numbers is $O(n\log{n})$.

##Sampling From a Distribution

Suppose that instead having all the values in our dataset, we just have a random subset of those values. We can generate a random sample (without replacement) of 100 observations from our heights values with the code below:

In [ ]:
heights_sample = np.random.choice(heights, size=100, replace=False)

In [ ]:
heights_sample

Now, this subsample can be viewed as a sample on its own, and we can calculate statistics for it like its mean and standard deviation.

In [ ]:
mean_value = np.mean(heights_sample)
std_dev_value = np.std(heights_sample)

print(f"Mean: {mean_value:.2f}")
print(f"Standard Deviation: {std_dev_value:.2f}")

What can this sample, and these sample statistics, tell us about the larger distribution? Well, if we look at the mean and standard deviation of our sample, we can note that it's pretty close to the mean and standard deviation of our larger dataset. As you might imagine, this isn't a coincidence.

Again, we're interested in machine learning, so we're interested in building models. Suppose we use the sample mean as a model of the actual mean. Is that a good model? How can we measure its performance?

Well, it turns out to be a very good estimate, in a very precise sense. Specifically:

* It's unbiased, which means that on average the sample mean will equal the mean of our larger dataset (commonly called the "population").
* It's the optimal unbiased estimator, in that of all unbiased estimators, it minimizes the variance. (Technically, it's the best *linear* unbiased estimator, but we won't get into that today.)

We can do a bit of exploratory investigation here, but taking 100 random samples of 100 observations, and checking out the distribution of the sample means. Note that this is a bit meta - we're looking at the distrubition of a statistic from samples of another distribution.

In [ ]:
averages_array = np.array([np.mean(np.random.choice(heights, size=100, replace=False)) for _ in range(100)])

If we plot these sample means on a histogram, it looks like:

In [ ]:
plt.hist(averages_array)
plt.xlabel('Height (cm)')
plt.ylabel('Frequency')
plt.title('Distribution of Sample Means from Galton Data')
plt.show()

mean_value = np.mean(averages_array)
median_value = np.median(averages_array)
std_dev_value = np.std(averages_array)

print(f"Mean: {mean_value:.2f}")
print(f"Median: {median_value:.2f}")
print(f"Standard Deviation: {std_dev_value:.2f}")

The mean of our sample means is quite close to the mean of the original distribution (the population). The median is also quite close to the mean of the original distribution. Nice.

Suppose we increased the number of samples from 100 to 1000.

In [ ]:
averages_array = np.array([np.mean(np.random.choice(heights, size=100, replace=False)) for _ in range(1000)])
plt.hist(averages_array, range=(65.5,68))
plt.xlabel('Height (cm)')
plt.ylabel('Frequency')
plt.title('Distribution of Sample Means from Galton Data')
plt.show()

mean_value = np.mean(averages_array)
median_value = np.median(averages_array)
std_dev_value = np.std(averages_array)

print(f"Mean: {mean_value:.2f}")
print(f"Median: {median_value:.2f}")
print(f"Standard Deviation: {std_dev_value:.2f}")

The mean and standard deviation don't change much, but the shape of the histogram is exhibiting some very interesting behavior. Let's increase the number of samples by another order of magnitude.

In [ ]:
averages_array = np.array([np.mean(np.random.choice(heights, size=100, replace=False)) for _ in range(10000)])
plt.hist(averages_array, range=(65.5,68))
plt.xlabel('Height (cm)')
plt.ylabel('Frequency')
plt.title('Distribution of Sample Means from Galton Data')
plt.show()

mean_value = np.mean(averages_array)
median_value = np.median(averages_array)
std_dev_value = np.std(averages_array)

print(f"Mean: {mean_value:.2f}")
print(f"Median: {median_value:.2f}")
print(f"Standard Deviation: {std_dev_value:.2f}")

It's looking more and more symmetric, and more and more regular. Let's go another OOM.

In [ ]:
averages_array = np.array([np.mean(np.random.choice(heights, size=100, replace=False)) for _ in range(100000)])
plt.hist(averages_array, range=(65.5,68))
plt.xlabel('Height (cm)')
plt.ylabel('Frequency')
plt.title('Distribution of Sample Means from Galton Data')
plt.show()

mean_value = np.mean(averages_array)
median_value = np.median(averages_array)
std_dev_value = np.std(averages_array)

print(f"Mean: {mean_value:.2f}")
print(f"Median: {median_value:.2f}")
print(f"Standard Deviation: {std_dev_value:.2f}")

That took a moment, and another OOM will take more than a moment, but perhaps by now you can see what's happening. With more and more samples, the mean and the standard deviation aren't changing, but the histogram is looking more and more like what we call a *normal distribution* (a.k.a. a *Gaussian distribution*, a.k.a. a *bell curve*.) The mathematical function for the normal distribution is:

<center>
  $\displaystyle p(x) = \frac{1}{\sigma\sqrt{2\pi}}e^{\frac{-(x-\mu)^{2}}{2\sigma^{2}}}$
</center>

Here $\mu$ is the mean, and $\sigma$ is the standard deviation. Note that these are the only two parameters here - the normal distribution is entirely determined by its mean and standard deviation (or, equivalently, its variance).

The mean and standard deviation didn't change much if we took more samples. What happens if we take larger samples? So, instead of 100 observations per sample, what if we have 200?

In [ ]:
averages_array = np.array([np.mean(np.random.choice(heights, size=200, replace=False)) for _ in range(10000)])
plt.hist(averages_array, range=(65.5,68))
plt.xlabel('Height (cm)')
plt.ylabel('Frequency')
plt.title('Distribution of Sample Means from Galton Data')
plt.show()

mean_value = np.mean(averages_array)
median_value = np.median(averages_array)
std_dev_value = np.std(averages_array)

print(f"Mean: {mean_value:.2f}")
print(f"Median: {median_value:.2f}")
print(f"Standard Deviation: {std_dev_value:.2f}")

Interesting! It appears mean doesn't change, but the standard deviation (the "spread") goes down. What happens if we increase the sample size again?

In [ ]:
averages_array = np.array([np.mean(np.random.choice(heights, size=300, replace=False)) for _ in range(10000)])
plt.hist(averages_array, range=(65.5,68))
plt.xlabel('Height (cm)')
plt.ylabel('Frequency')
plt.title('Distribution of Sample Means from Galton Data')
plt.show()

mean_value = np.mean(averages_array)
median_value = np.median(averages_array)
std_dev_value = np.std(averages_array)

print(f"Mean: {mean_value:.2f}")
print(f"Median: {median_value:.2f}")
print(f"Standard Deviation: {std_dev_value:.2f}")

Whoa! It's even smaller. Let's do one more.

In [ ]:
averages_array = np.array([np.mean(np.random.choice(heights, size=400, replace=False)) for _ in range(10000)])
plt.hist(averages_array, range=(65.5,68))
plt.xlabel('Height (cm)')
plt.ylabel('Frequency')
plt.title('Distribution of Sample Means from Galton Data')
plt.show()

mean_value = np.mean(averages_array)
median_value = np.median(averages_array)
std_dev_value = np.std(averages_array)

print(f"Mean: {mean_value:.2f}")
print(f"Median: {median_value:.2f}")
print(f"Standard Deviation: {std_dev_value:.2f}")

It seems to be a pattern - and it is.

These observations all stem from the *central limit theorem*, which could be viewed as the foundational result in statistics.

###The Central Limit Theorem

I won't state the central limit theorem in its full mathematical generality here, but instead I'll put it in terms most relevant to the question at hand. The central limit theorem says that the statistical distribution of the means for random samples from a population is a normal distribution, with mean equal to the population mean, and standard deviation determined by the population's standard deviation *and* the size of the sample. More precisely, the standard deviation for the sample means is $\sigma / \sqrt{n}$, where $\sigma$ is the standard deviation of the population, and $n$ is the size of the sample.

*Note* - The size of the population doesn't actually come into play here! So, as long as the population is larger than the sample, which of course it would need to be, the population size isn't relevant. This is why it's possible to get pretty good mean estimates with a sample size of just a few hundred even if the population size is millions!

### Confidence Intervals

Let's go back to the single sample from earlier in the lecture:

In [ ]:
heights_sample

The mean of this sample is:

In [ ]:
np.mean(heights_sample)

While, on average, this sample mean will be equal to the population mean, we of course can't expect it to exactly equal the population mean. There's some uncertainty. Can we quantify this uncertainty? That is to say, instead of this point estimate, can we provide a range within which we're reasonably sure we'll find the population mean?

Yes! We first need to choose what we mean by "reasonably sure". Typically it's 95%, but there's no profound physical or mathematical reason why humans have decided 95% - as compared to, say, 94% or 96% - is the proper measure for truth. It may have more to do with the number of fingers we have than anything else.

Here's how we do it. We first need to define what percentage we want for our confidence interval. It could by 80%, 90%, 95%, 99%, 99.9% - whatever we want. The choice might depend on some additional calculations, or it might just be picked arbitrarily. Does matter, but we need some number between 0% and 100%. Call it $P$.

This percentage gives us the *critical* value from the standard normal distribution corresponding to the desired confidence interval. This is usually denoted by $z^{*}$, and basically just means how many standard deviations away from the mean do we need to be in order for $P% of the normal distribution to be within the interval. For 95% it's about 1.96.

Our confidence interval is then given by:

<center>
  $\displaystyle CI = \left(\overline{X}-z^{*}\cdot\frac{\sigma}{\sqrt{n}},\overline{X}+z^{*}\cdot\frac{\sigma}{\sqrt{n}}\right)$
</center>

Here $\overline{X}$ is out sample mean, $\sigma$ is our sample standard deviation, and $n$ is the size of our sample. This can be interpreted as saying that, if we're sampling randomly, 95% of the time the true population mean will fall within the confidence interval of our sample.

##Hypothesis tests, p-values, and t-statistics

Let's go back to our height datafram.

In [ ]:
df.head()

Suppose I wanted to use this data to investigate some questions regarding the means of subpopulations. Specifically, let's suppose I want to investigate:

* Whether the average height of males is different than the average height of females.
* Whether the average height of first children is greaten than the average height of second children.

I think we'd all expect the answer to the first to be yes, but would be surprised if this were the answer to the second.

One thing we would not be surprised at is if you sampled both subpopulations, and found the subsamples had different means. Because of the nature of randomness we'd expect the subsamples to not have identical means. So, how do we tell if the difference is "real" (the subpopulations actually have different means), or just a random result of the subsamples. This is where you need to formulate a *hypothesis test*.

For our hypothesis test, we first define a *null hypothesis*, which means the state where our hypothesis is wrong. So, the null hypothesis for our gender question above is that, on average, the height of males is the same as the heigh of females. This is typically denoted $H_{0}$. More specifically, we'll assume they have the same distribution (or, more technically, that their distributions have the same mean and the same variance). In this case, there is a well known distribution for something called the "t-statistic", which is defined as:

<center>
  $\displaystyle t = \frac{\overline{X}_{1} - \overline{X}_{2}}{\sqrt{\frac{s_{1}^{2}}{n_{1}} + \frac{s_{2}^{2}}{n_{2}}}}$
</center>

Here $X_{1}$, $s_{1}$, and $n_{1}$ are the mean, standard deviation, and sample size of the first sample, while $X_{2}$, $s_{2}$, and $n_{2}$ are the mean, standard deviation, and samples size of the second sample. If the null hypothesis is correct, then it's unlikely that the absolute value of $t$ will be very large. In fact, we can precisely provide bounds that say, for example, than 90% of the time the $t$-statistic will be within some interval around $0$. So, how do we "reject" the null hypothesis? Well, if the $t$ value is too large, then we say it's very unlikely the null hypothesis is true, and so we reject it. What's the criterion for rejection? Again, that's up to you! You can say you reject at 90%, or at 95%, or at 99%, or at 99.9%. Nature provides no value for which you can assume a satisfactory level of certainty. You must choose that yourself.

The *stats* module from the *scipy* library provides some nice functions for doing this sort of hypothesis testing. Let's first create our subpopulations.

In [ ]:
males = df[df['gender'] == 'male']
females = df[df['gender'] == 'female']

first_borns = df[df['childNum'] == 1]
second_borns = df[df['childNum'] == 2]

Now, let's sample from them. We'll randomly sample 100 from each.

In [ ]:
males_sample = np.random.choice(males['childHeight'], size=100, replace=False)
females_sample = np.random.choice(females['childHeight'], size=100, replace=False)

first_borns_sample = np.random.choice(first_borns['childHeight'], size=100, replace=False)
second_borns_sample = np.random.choice(second_borns['childHeight'], size=100, replace=False)

Let's check out the means of each sample.

In [ ]:
print(f"The mean male height is: {np.mean(males_sample):.2f}")
print(f"The mean female height is: {np.mean(females_sample):.2f}")
print(f"The mean first born height is: {np.mean(first_borns_sample):.2f}")
print(f"The mean second born height is: {np.mean(second_borns_sample):.2f}")

It looks like there's quite a difference between the male and female heights, and a smaller one - but maybe not negligibly small - between the first and second born heights.

Can we reject the null hypothesis that males and females are, on average, the same height?

In [ ]:
t_stat, p_value = stats.ttest_ind(males_sample, females_sample)

print(f"t-statistic: {t_stat}")
print(f"p-value: {p_value}")

That $p$-value is *extremely* small, which means that if the null hypothesis is true, the probability of the mean difference we observe is *very* small. We should probably reject. On the other hand, what is it for the first and second born samples?

In [ ]:
t_stat, p_value = stats.ttest_ind(first_borns_sample, second_borns_sample)

print(f"t-statistic: {t_stat}")
print(f"p-value: {p_value}")

Interesting... this is saying that if the null hypothesis is correct (that there's no difference in average heights for first and second born children) that we'd see a difference as large as (or larger than) we did only about 1% of the time. Maybe being born first is an advantage? This could warrant further study, which I'll leave to you.

## References

* Statquest of [Hypothesis Testing](https://youtu.be/0oc49DyA3hU?si=MN24V0vmMP1AlrTi)

* The [Hypothesis Testing Rap](https://youtu.be/SrK5UCOYm6U?si=YWZW-9w2x45TWKxC)